In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [2]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result
0,36941,ESPORTSTMNT01_2698440,20,21,0,24,20,23,21,22,...,6,0.00,0.5,0,0.00,0.0,2,0.50,8.7,1
1,36960,ESPORTSTMNT06_2210164,0,21,1,22,5,2,24,25,...,0,0.00,0.0,8,1.00,14.6,1,1.00,15.0,0
2,36961,ESPORTSTMNT06_2210170,19,2,20,3,5,1,21,22,...,54,0.57,3.5,2,1.00,4.3,9,0.22,1.8,0
3,36962,ESPORTSTMNT06_2190198,19,2,3,20,24,4,20,2,...,8,0.50,4.6,1,1.00,12.0,2,0.50,6.0,0
4,36963,ESPORTSTMNT06_2210177,0,1,2,3,5,1,2,8,...,21,0.57,4.6,3,0.33,3.7,31,0.61,5.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,42468,ESPORTSTMNT06_2531063,20,0,2,1,5,4,8,1,...,5,1.00,12.5,14,0.29,2.7,19,0.63,4.7,1
1336,42469,ESPORTSTMNT06_2531053,0,2,20,4,3,2,1,5,...,0,0.00,0.0,1,0.00,0.3,2,0.00,0.8,0
1337,42470,ESPORTSTMNT06_2531038,19,20,21,20,5,3,24,2,...,1,0.00,1.0,4,0.50,4.4,3,0.00,3.1,0
1338,42471,ESPORTSTMNT06_2541067,19,20,2,24,23,4,20,3,...,0,0.00,0.0,0,0.00,0.0,0,0.00,0.0,1


In [3]:
y = df['result'].copy()
X = df.drop(['result','game','event6','event7','event8','event9','event10','event11','event12','event13','event14','event15','event16','event17','event18','event19','event20','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X

,golId,event1,event2,event3,event4,event5,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,36941,20,21,0,24,20,0,0.00,0.0,0,...,1.6,6,0.00,0.5,0,0.00,0.0,2,0.50,8.7
1,36960,0,21,1,22,5,1,0.00,1.7,16,...,4.0,0,0.00,0.0,8,1.00,14.6,1,1.00,15.0
2,36961,19,2,20,3,5,2,1.00,7.0,5,...,6.2,54,0.57,3.5,2,1.00,4.3,9,0.22,1.8
3,36962,19,2,3,20,24,20,0.35,2.1,52,...,4.8,8,0.50,4.6,1,1.00,12.0,2,0.50,6.0
4,36963,0,1,2,3,5,12,0.50,2.4,4,...,1.5,21,0.57,4.6,3,0.33,3.7,31,0.61,5.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,42468,20,0,2,1,5,13,0.54,3.7,5,...,1.7,5,1.00,12.5,14,0.29,2.7,19,0.63,4.7
1336,42469,0,2,20,4,3,12,0.42,3.7,6,...,0.5,0,0.00,0.0,1,0.00,0.3,2,0.00,0.8
1337,42470,19,20,21,20,5,15,0.67,2.6,7,...,2.1,1,0.00,1.0,4,0.50,4.4,3,0.00,3.1
1338,42471,19,20,2,24,23,0,0.00,0.0,0,...,4.7,0,0.00,0.0,0,0.00,0.0,0,0.00,0.0


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained


c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [6]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [7]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.686567,0.678832,0.699248,0.688889
1,Support Vector Machine (Linear Kernel),0.686567,0.678832,0.699248,0.688889
2,Support Vector Machine (RBF Kernel),0.701493,0.687943,0.729323,0.708029
3,Decission Tree,0.597015,0.586207,0.639098,0.611511
4,Adaboost,0.664179,0.659259,0.669173,0.664179
5,Random Forest,0.697761,0.691176,0.706767,0.698885
6,Gradient Boosting Classifier,0.690299,0.689394,0.684211,0.686792
